### AutoGen Vs Microsoft Agent Framework Migration 📌

#### Single-Agent 🤖

E-commerce Order Management Assistant
Comparison: `AutoGen` vs `Agent Framework`

Use Case: Customer support agent that can:
1. Look up order status
2. Process refunds
3. Update shipping addresses
4. Handle multi-turn conversations

This demonstrates:
- AutoGen: Stateful multi-turn conversations with context
- Agent Framework: Lightweight stateless interactions


In [ ]:
# Install all required packages
! pip install autogen-agentchat autogen-ext[openai] agent-framework openai python-dotenv -q
print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [2]:
# Import all required libraries
import asyncio
import json
import os
from typing import Annotated
from datetime import datetime
from dotenv import load_dotenv

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [3]:
load_dotenv()

True

In [4]:
# Configure API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

print("✅ API Key configured!")

✅ API Key configured!


In [5]:
# Create mock e-commerce database

# Mock order database
ORDERS_DB = {
    "ORD12345": {
        "id": "ORD12345",
        "customer": "John Doe",
        "items": ["Laptop", "Mouse"],
        "total": 1299.99,
        "status": "shipped",
        "tracking": "TRACK123456"
    },
    "ORD67890": {
        "id": "ORD67890",
        "customer": "Jane Smith",
        "items": ["Phone Case"],
        "total": 29.99,
        "status": "delivered",
        "tracking": "TRACK789012"
    },
    "ORD11111": {
        "id": "ORD11111",
        "customer": "Bob Wilson",
        "items": ["Headphones", "Cable"],
        "total": 199.99,
        "status": "processing",
        "tracking": None
    }
}

print("✅ Mock database created!")
print(f"📦 {len(ORDERS_DB)} orders in database")


✅ Mock database created!
📦 3 orders in database


In [6]:
# Define shared business functions (used by both frameworks)

def lookup_order(order_id: str) -> str:
    """Look up order details by order ID."""
    print(f"🔍 Looking up order: {order_id}")
    
    order = ORDERS_DB.get(order_id)
    if not order:
        return json.dumps({"error": f"Order {order_id} not found"})
    
    return json.dumps({
        "order_id": order["id"],
        "customer": order["customer"],
        "items": order["items"],
        "total": order["total"],
        "status": order["status"],
        "tracking": order["tracking"]
    }, indent=2)


def process_refund(order_id: str, amount: float, reason: str) -> str:
    """Process a refund for an order."""
    print(f"💰 Processing refund: ${amount} for {order_id}")
    
    order = ORDERS_DB.get(order_id)
    if not order:
        return json.dumps({"error": f"Order {order_id} not found"})
    
    if amount > order["total"]:
        return json.dumps({"error": f"Refund amount ${amount} exceeds order total ${order['total']}"})
    
    refund_id = f"REF{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    return json.dumps({
        "status": "success",
        "refund_id": refund_id,
        "order_id": order_id,
        "amount": amount,
        "reason": reason,
        "processed_at": datetime.now().isoformat()
    }, indent=2)


def update_shipping_address(order_id: str, new_address: str) -> str:
    """Update shipping address for an order."""
    print(f"📦 Updating shipping address for: {order_id}")
    
    order = ORDERS_DB.get(order_id)
    if not order:
        return json.dumps({"error": f"Order {order_id} not found"})
    
    if order["status"] == "delivered":
        return json.dumps({"error": "Cannot update address for delivered order"})
    
    return json.dumps({
        "status": "success",
        "order_id": order_id,
        "new_address": new_address,
        "updated_at": datetime.now().isoformat()
    }, indent=2)


def cancel_order(order_id: str) -> str:
    """Cancel an order if not yet shipped."""
    print(f"❌ Attempting to cancel order: {order_id}")
    
    order = ORDERS_DB.get(order_id)
    if not order:
        return json.dumps({"error": f"Order {order_id} not found"})
    
    if order["status"] in ["shipped", "delivered"]:
        return json.dumps({"error": f"Cannot cancel order with status: {order['status']}"})
    
    return json.dumps({
        "status": "cancelled",
        "order_id": order_id,
        "cancelled_at": datetime.now().isoformat(),
        "refund_amount": order["total"]
    }, indent=2)


print("✅ Business functions defined!")
print("   • lookup_order")
print("   • process_refund")
print("   • update_shipping_address")
print("   • cancel_order")


✅ Business functions defined!
   • lookup_order
   • process_refund
   • update_shipping_address
   • cancel_order


In [7]:
# AutoGen - Stateful Multi-Turn Conversation

async def run_autogen_multiturn():
    """
    AutoGen Implementation: Multi-turn conversation with automatic context.
    Demonstrates AutoGen's strength in maintaining conversation state.
    """
    from autogen_agentchat.agents import AssistantAgent
    from autogen_ext.models.openai import OpenAIChatCompletionClient
    
    print("\n" + "="*70)
    print("🤖 AUTOGEN - STATEFUL MULTI-TURN CONVERSATION")
    print("="*70)
    
    # Create model client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    )
    
    # Create assistant agent with tools
    assistant = AssistantAgent(
        name="customer_support",
        model_client=model_client,
        tools=[lookup_order, process_refund, update_shipping_address, cancel_order],
        system_message="""You are a helpful customer support agent for an e-commerce company.

You can:
1. Look up order status and details
2. Process refunds (up to order amount)
3. Update shipping addresses (only for non-delivered orders)
4. Cancel orders (only if not shipped yet)

Always:
- Be empathetic and professional
- Look up order details before processing any changes
- Confirm amounts and details before executing actions
- Explain what you're doing step by step
- Ask for missing information

For refunds, always confirm the amount and reason first."""
    )
    
    # Multi-turn conversation scenario
    conversation = [
        "Hi, I need help with order ORD12345",
        "I want to return it because the laptop arrived with a scratch",
        "Yes, please process a full refund",
        "Thank you! Can you also tell me the status of order ORD67890?"
    ]
    
    print("\n📞 Customer Support Conversation:\n")
    
    # AutoGen maintains state across multiple interactions automatically
    for i, user_message in enumerate(conversation, 1):
        print(f"👤 Customer (Turn {i}): {user_message}")
        
        # Run agent - AutoGen automatically remembers previous context
        result = await assistant.run(task=user_message)
        
        # Get the last assistant message
        agent_response = result.messages[-1].content
        print(f"🤖 Agent: {agent_response}\n")
        
        await asyncio.sleep(0.5)  # Simulate thinking time
    
    print("✅ AutoGen Strengths Demonstrated:")
    print("   ✓ Automatic conversation context maintenance")
    print("   ✓ Natural follow-up questions work seamlessly")
    print("   ✓ No manual state management needed")
    print("   ✓ References to 'it' and 'the order' understood from context")
    print("   ✓ Tool execution with full conversation awareness")


print("✅ AutoGen multi-turn function defined!")
print("🚀 Run with: await run_autogen_multiturn()")


✅ AutoGen multi-turn function defined!
🚀 Run with: await run_autogen_multiturn()


In [ ]:
# Agent Framework - Implementation

async def run_agent_framework_multiturn():
    """
    Agent Framework Implementation: Stateless design.
    NOTE: Agent Framework ChatAgent is designed for single-turn interactions.
    For multi-turn, you need to use a different pattern.
    """
    from agent_framework import ChatAgent
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("⚡ AGENT FRAMEWORK - STATELESS SINGLE-TURN PATTERN")
    print("="*70)
    
    # Create chat client and agent
    client = OpenAIChatClient()
    agent = ChatAgent(
        chat_client=client,
        instructions="""You are a helpful customer support agent for an e-commerce company.

You can:
1. Look up order status and details
2. Process refunds (up to order amount)
3. Update shipping addresses (only for non-delivered orders)
4. Cancel orders (only if not shipped yet)

Always:
- Be empathetic and professional
- Look up order details before processing any changes
- Confirm amounts and details before executing actions
- Explain what you're doing step by step""",
        tools=[lookup_order, process_refund, update_shipping_address, cancel_order],
        model="gpt-4o-mini"
    )
    
    conversation = [
        "Hi, I need help with order ORD12345",
        "I want to return it because the laptop arrived with a scratch",
        "Yes, please process a full refund for $1299.99",
        "Thank you! Can you also tell me the status of order ORD67890?"
    ]
    
    print("\n📞 Customer Support Conversation:")
    print("⚠️  NOTE: Each query is independent - no automatic context!\n")
    
    # Agent Framework: Each call is independent (stateless)
    for i, user_message in enumerate(conversation, 1):
        print(f"👤 Customer (Turn {i}): {user_message}")
        
        # Each run is independent - no context from previous turns
        result = await agent.run(user_message)
        
        print(f"⚡ Agent: {result.text}\n")
        
        await asyncio.sleep(0.5)
    
    print("="*70)
    print("⚠️  IMPORTANT OBSERVATION:")
    print("="*70)
    print("Notice how the agent doesn't remember previous context!")
    print("• Turn 2: Doesn't know which order (no 'ORD12345' context)")
    print("• Turn 3: Doesn't know the refund is for the laptop order")
    print("• Each turn is COMPLETELY INDEPENDENT")
    
    print("\n✅ This demonstrates Agent Framework's stateless design:")
    print("   • Each request is independent")
    print("   • No automatic context retention")
    print("   • Perfect for single-turn API calls")
    print("   • Not ideal for multi-turn conversations WITHOUT external state")


print("✅ Corrected Agent Framework function defined!")
print("🚀 Run with: await run_agent_framework_multiturn()")


✅ Corrected Agent Framework function defined!
🚀 Run with: await run_agent_framework_multiturn()


In [ ]:
# Single-Turn Query Comparison

async def compare_single_turn():
    """
    Single-turn queries comparison.
    Shows where Agent Framework's simplicity shines.
    """
    print("\n" + "="*70)
    print("⚡ SINGLE-TURN QUERY COMPARISON")
    print("="*70)
    print("\nScenario: Quick order status lookup (no follow-up needed)\n")
    
    query = "What's the status of order ORD12345?"
    
    # ========== AUTOGEN ==========
    print("📦 AutoGen Approach:")
    print("-" * 70)
    
    from autogen_agentchat.agents import AssistantAgent
    from autogen_ext.models.openai import OpenAIChatCompletionClient
    
    client_ag = OpenAIChatCompletionClient(model="gpt-4o-mini")
    agent_ag = AssistantAgent(
        name="support",
        model_client=client_ag,
        tools=[lookup_order],
        system_message="You are a support agent. Look up the order and report status clearly."
    )
    
    print(f"Query: {query}")
    result_ag = await agent_ag.run(task=query)
    print(f"Response: {result_ag.messages[-1].content}\n")
    
    # ========== AGENT FRAMEWORK ==========
    print("📦 Agent Framework Approach:")
    print("-" * 70)
    
    from agent_framework import ChatAgent
    from agent_framework.openai import OpenAIChatClient
    
    client_af = OpenAIChatClient()
    agent_af = ChatAgent(
        chat_client=client_af,
        instructions="You are a support agent. Look up the order and report status clearly.",
        tools=[lookup_order],
        model="gpt-4o-mini"
    )
    
    print(f"Query: {query}")
    result_af = await agent_af.run(query)
    print(f"Response: {result_af.text}\n")
    
    # ========== COMPARISON ==========
    print("="*70)
    print("📊 Analysis for Single-Turn Queries:")
    print("="*70)
    print("\n✅ Agent Framework Advantages:")
    print("   • Simpler API (just pass string)")
    print("   • Less setup code")
    print("   • Faster execution (no state overhead)")
    print("   • Direct text response (.text property)")
    print("   • Perfect for REST APIs and serverless")
    
    print("\n⚠️  AutoGen Considerations:")
    print("   • More structured (task parameter)")
    print("   • Message objects add complexity")
    print("   • Maintains state even when not needed")
    print("   • Better if you might add follow-ups later")


print("✅ Single-turn comparison function defined!")
print("🚀 Run with: await compare_single_turn()")


✅ Single-turn comparison function defined!
🚀 Run with: await compare_single_turn()


In [10]:
# Run AutoGen Multi-Turn Demo
await run_autogen_multiturn()


🤖 AUTOGEN - STATEFUL MULTI-TURN CONVERSATION

📞 Customer Support Conversation:

👤 Customer (Turn 1): Hi, I need help with order ORD12345
🔍 Looking up order: ORD12345
🤖 Agent: {
  "order_id": "ORD12345",
  "customer": "John Doe",
  "items": [
    "Laptop",
    "Mouse"
  ],
  "total": 1299.99,
  "status": "shipped",
  "tracking": "TRACK123456"
}

👤 Customer (Turn 2): I want to return it because the laptop arrived with a scratch
🤖 Agent: I'm sorry to hear that your laptop arrived with a scratch. I can help you process a return for this order. 

Before we proceed, I would like to confirm a few details:

1. The order total is $1299.99. Is this the amount you would like to initiate a refund for?
2. Can you please confirm that the reason for the refund is that the laptop arrived with a scratch?

Once I have this information, I can begin processing your return.

👤 Customer (Turn 3): Yes, please process a full refund
💰 Processing refund: $1299.99 for ORD12345
🤖 Agent: {
  "status": "success",


In [15]:
# Run Agent Framework Multi-Turn Demo
await run_agent_framework_multiturn()


⚡ AGENT FRAMEWORK - STATELESS SINGLE-TURN PATTERN

📞 Customer Support Conversation:
⚠️  NOTE: Each query is independent - no automatic context!

👤 Customer (Turn 1): Hi, I need help with order ORD12345
🔍 Looking up order: ORD12345
⚡ Agent: I found the details for your order ORD12345.

- **Customer Name:** John Doe
- **Items:** Laptop, Mouse
- **Total Amount:** $1299.99
- **Status:** Shipped
- **Tracking Number:** TRACK123456

Since your order has already been shipped, I won't be able to update the shipping address or cancel the order. However, I can assist you with any other concerns related to this order. How can I help you today?

👤 Customer (Turn 2): I want to return it because the laptop arrived with a scratch
⚡ Agent: I'm sorry to hear that your laptop arrived with a scratch. I can certainly assist you with processing a refund. 

To proceed, I will need to look up your order details first. Please provide me with your order ID.

👤 Customer (Turn 3): Yes, please process a full refu

In [16]:
# Run Single-Turn Comparison
await compare_single_turn()



⚡ SINGLE-TURN QUERY COMPARISON

Scenario: Quick order status lookup (no follow-up needed)

📦 AutoGen Approach:
----------------------------------------------------------------------
Query: What's the status of order ORD12345?
🔍 Looking up order: ORD12345
Response: {
  "order_id": "ORD12345",
  "customer": "John Doe",
  "items": [
    "Laptop",
    "Mouse"
  ],
  "total": 1299.99,
  "status": "shipped",
  "tracking": "TRACK123456"
}

📦 Agent Framework Approach:
----------------------------------------------------------------------
Query: What's the status of order ORD12345?
🔍 Looking up order: ORD12345
Response: The status of order **ORD12345** is **shipped**. 

Here are the details:
- **Customer:** John Doe
- **Items:** Laptop, Mouse
- **Total Amount:** $1299.99
- **Tracking Number:** TRACK123456

If you need any further assistance, feel free to ask!

📊 Analysis for Single-Turn Queries:

✅ Agent Framework Advantages:
   • Simpler API (just pass string)
   • Less setup code
   • Faster

In [ ]:
# Print comprehensive decision guide

def print_decision_guide():
    """Print when to use each framework."""
    guide = """
╔═══════════════════════════════════════════════════════════════════════════╗
║                    WHEN TO USE EACH FRAMEWORK                             ║
╚═══════════════════════════════════════════════════════════════════════════╝

🤖 USE AUTOGEN WHEN:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Building conversational AI with context
✅ Multi-turn dialogues where history matters
✅ Complex multi-agent systems (agents talking to agents)
✅ Research and experimentation with agent behaviors
✅ Need automatic state management
✅ Team-based workflows with multiple agents
✅ Educational chatbots or tutoring systems

📋 Best Use Cases:
   • Customer support chatbots (like our demo)
   • Virtual assistants with memory
   • Multi-agent research systems
   • Complex troubleshooting workflows
   • Interactive tutoring systems
   • Collaborative agent teams


⚡ USE AGENT FRAMEWORK WHEN:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Building REST APIs or microservices
✅ Single-turn or simple queries
✅ Need explicit control over state
✅ Serverless functions (AWS Lambda, Azure Functions)
✅ High-performance requirements
✅ Minimal abstraction preferred
✅ Integration with existing systems

📋 Best Use Cases:
   • API endpoints for order lookup
   • Serverless automation functions
   • Simple Q&A systems
   • Database query interfaces
   • Slack/Discord bot commands
   • Quick automation tasks
   • Stateless microservices


🎯 REAL-WORLD SCENARIOS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Scenario 1: Customer Support Chat
   Choice: AUTOGEN ✅
   Reason: Multi-turn, context-dependent, follow-up questions

Scenario 2: API Endpoint /api/order-status
   Choice: AGENT FRAMEWORK ✅
   Reason: Single request/response, stateless, fast

Scenario 3: Slack Bot for Simple Commands
   Choice: AGENT FRAMEWORK ✅
   Reason: Each command is independent, no conversation flow

Scenario 4: Virtual Teaching Assistant
   Choice: AUTOGEN ✅
   Reason: Long conversations, needs to remember student progress

Scenario 5: Database Query Interface
   Choice: AGENT FRAMEWORK ✅
   Reason: Independent queries, no conversation context needed

Scenario 6: Multi-Agent Research System
   Choice: AUTOGEN ✅
   Reason: Agents need to collaborate, share context


📊 FEATURE COMPARISON TABLE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Feature                    | AutoGen        | Agent Framework
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Conversation Memory        | ✅ Automatic   | ⚠️  Manual
State Management           | ✅ Built-in    | ⚠️  External
Multi-Turn Conversations   | ✅ Native      | ⚠️  Manual work
Single-Turn Queries        | ⚠️  Overkill   | ✅ Perfect
Learning Curve             | ⚠️  Moderate   | ✅ Low
API Simplicity             | ⚠️  Structured | ✅ Simple
Performance (Stateless)    | ⚠️  Overhead   | ✅ Fast
Multi-Agent Teams          | ✅ Excellent   | ⚠️  Custom
REST API Development       | ⚠️  Heavy      | ✅ Ideal
Serverless Functions       | ⚠️  Too much   | ✅ Perfect
Research & Experimentation | ✅ Great       | ⚠️  Basic
Production Ready           | ✅ Yes         | ✅ Yes
Context Awareness          | ✅ Excellent   | ⚠️  Manual
Tool/Function Calling      | ✅ Rich        | ✅ Simple
Message History            | ✅ Automatic   | 📝 You manage
State Persistence          | 💾 In-memory   | 💾 Your choice


💡 HYBRID APPROACH:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
You can use BOTH in the same project:

1. Agent Framework for simple, stateless endpoints
   Example: GET /api/order/{id}/status

2. AutoGen for complex, conversational flows
   Example: POST /api/chat (with session management)

3. Best of both worlds:
   • Fast API responses with Agent Framework
   • Rich conversations with AutoGen when needed


🔑 KEY DECISION FACTORS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Ask yourself:

❓ "Do I need to remember conversation context?"
   YES → AutoGen
   NO → Agent Framework

❓ "Is this a multi-turn conversation?"
   YES → AutoGen
   NO → Agent Framework

❓ "Do I need multiple agents collaborating?"
   YES → AutoGen
   NO → Agent Framework

❓ "Is this a simple API endpoint?"
   YES → Agent Framework
   NO → Maybe AutoGen

❓ "Do I want explicit control over state?"
   YES → Agent Framework
   NO → AutoGen


✅ CONCLUSION:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Both frameworks are excellent and production-ready!

Choose based on:
   • State management needs
   • Conversation complexity
   • Performance requirements
   • Architecture preferences

Don't overthink it - you can always switch or use both! 🚀
"""
    print(guide)

print_decision_guide()



╔═══════════════════════════════════════════════════════════════════════════╗
║                    WHEN TO USE EACH FRAMEWORK                             ║
╚═══════════════════════════════════════════════════════════════════════════╝

🤖 USE AUTOGEN WHEN:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Building conversational AI with context
✅ Multi-turn dialogues where history matters
✅ Complex multi-agent systems (agents talking to agents)
✅ Research and experimentation with agent behaviors
✅ Need automatic state management
✅ Team-based workflows with multiple agents
✅ Educational chatbots or tutoring systems

📋 Best Use Cases:
   • Customer support chatbots (like our demo)
   • Virtual assistants with memory
   • Multi-agent research systems
   • Complex troubleshooting workflows
   • Interactive tutoring systems
   • Collaborative agent teams


⚡ USE AGENT FRAMEWORK WHEN:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ Bui

In [ ]:
# Show architecture differences

def show_architecture_comparison():
    """Visual comparison of architectures."""
    
    architecture = """
╔═══════════════════════════════════════════════════════════════════════════╗
║                    ARCHITECTURE COMPARISON                                ║
╚═══════════════════════════════════════════════════════════════════════════╝


🤖 AUTOGEN ARCHITECTURE (Stateful)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    User Request 1
         │
         ▼
    ┌─────────────────┐
    │  AssistantAgent │ ◄─── Maintains internal state
    │  (with tools)   │
    └────────┬────────┘
             │
    ┌────────▼────────┐
    │ Message History │ ◄─── Automatically stored
    │  [msg1, msg2]   │
    └────────┬────────┘
             │
    User Request 2 ───┘ (Has context from Request 1)
         │
         ▼
    Response 2 (understands "it", "the order", etc.)


⚡ AGENT FRAMEWORK ARCHITECTURE (Stateless)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    User Request 1
         │
         ▼
    ┌─────────────────┐
    │   ChatAgent     │ ◄─── Stateless, no memory
    │  (with tools)   │
    └────────┬────────┘
             │
             ▼
         Response 1
    
    ┌────────────────┐
    │ External Store │ ◄─── YOU manage state
    │ (DB/Redis/etc) │      (if needed)
    └────────┬───────┘
             │
    User Request 2 + History ───┐
                                │
                                ▼
                        ┌─────────────────┐
                        │   ChatAgent     │
                        │  (with tools)   │
                        └────────┬────────┘
                                 │
                                 ▼
                            Response 2


🔄 DATA FLOW COMPARISON
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

AutoGen (Automatic):
    Request → Agent (remembers) → Response
              ↓
         [State stays in agent]

Agent Framework (Manual):
    Request → Agent (stateless) → Response
         ↓                              ↓
    [Store in DB/Redis]  ←───  [Return + save]
         ↓
    Next Request + History → Agent → Response


💾 STATE PERSISTENCE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

AutoGen:
    • State in memory by default
    • Can serialize/deserialize for persistence
    • Best for single-server deployments

Agent Framework:
    • You choose where to store state:
      - PostgreSQL
      - Redis
      - DynamoDB
      - Cosmos DB
    • Perfect for distributed systems


🏗️ MICROSERVICES PATTERN
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Agent Framework (Ideal):

    ┌──────────────┐
    │   API 1      │
    │ (Agent Fwk)  │ ◄─── GET /order-status
    └──────────────┘

    ┌──────────────┐
    │   API 2      │
    │ (Agent Fwk)  │ ◄─── POST /process-refund
    └──────────────┘

    ┌──────────────┐
    │   API 3      │
    │ (Agent Fwk)  │ ◄─── PUT /update-address
    └──────────────┘

Each service is independent, lightweight, fast


AutoGen (For Complex Flows):

    ┌────────────────────────────┐
    │  Conversation Manager      │
    │  (AutoGen Team/GroupChat)  │
    └──────────┬─────────────────┘
               │
       ┌───────┴──────┐
       ▼              ▼
    Agent 1        Agent 2
    (Orders)       (Refunds)
       │              │
       └──────┬───────┘
              ▼
         Coordinator
            Agent
"""
    print(architecture)

show_architecture_comparison()



╔═══════════════════════════════════════════════════════════════════════════╗
║                    ARCHITECTURE COMPARISON                                ║
╚═══════════════════════════════════════════════════════════════════════════╝


🤖 AUTOGEN ARCHITECTURE (Stateful)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    User Request 1
         │
         ▼
    ┌─────────────────┐
    │  AssistantAgent │ ◄─── Maintains internal state
    │  (with tools)   │
    └────────┬────────┘
             │
    ┌────────▼────────┐
    │ Message History │ ◄─── Automatically stored
    │  [msg1, msg2]   │
    └────────┬────────┘
             │
    User Request 2 ───┘ (Has context from Request 1)
         │
         ▼
    Response 2 (understands "it", "the order", etc.)


⚡ AGENT FRAMEWORK ARCHITECTURE (Stateless)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    User Request 1
         │
         ▼
    ┌─────────────────┐
    │   ChatAgent 

In [19]:
# Run all comparisons in sequence

async def run_complete_comparison():
    """Run all comparison demos."""
    
    print("\n" + "🎯"*35)
    print("COMPLETE AUTOGEN VS AGENT FRAMEWORK COMPARISON")
    print("Use Case: E-commerce Customer Support")
    print("🎯"*35)
    
    # 1. AutoGen multi-turn
    await run_autogen_multiturn()
    
    input("\n⏸️  Press Enter to continue to Agent Framework demo...")
    
    # 2. Agent Framework multi-turn
    await run_agent_framework_multiturn()
    
    input("\n⏸️  Press Enter to continue to single-turn comparison...")
    
    # 3. Single-turn comparison
    await compare_single_turn()
    
    # 4. Print guides
    print("\n")
    print_decision_guide()
    
    print("\n")
    show_architecture_comparison()
    
    print("\n" + "="*70)
    print("✅ COMPARISON COMPLETE!")
    print("="*70)
    print("\n💡 Key Takeaways:")
    print("   1. AutoGen: Best for STATEFUL, MULTI-TURN conversations")
    print("   2. Agent Framework: Best for STATELESS, SIMPLE operations")
    print("   3. Both are production-ready and excellent")
    print("   4. Choose based on your STATE MANAGEMENT needs")
    print("   5. You can use BOTH in the same project!\n")

print("✅ Complete comparison function ready!")
print("🚀 Run with: await run_complete_comparison()")


✅ Complete comparison function ready!
🚀 Run with: await run_complete_comparison()


In [20]:
# Execute the complete comparison
await run_complete_comparison()


🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
COMPLETE AUTOGEN VS AGENT FRAMEWORK COMPARISON
Use Case: E-commerce Customer Support
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯

🤖 AUTOGEN - STATEFUL MULTI-TURN CONVERSATION

📞 Customer Support Conversation:

👤 Customer (Turn 1): Hi, I need help with order ORD12345
🔍 Looking up order: ORD12345
🤖 Agent: {
  "order_id": "ORD12345",
  "customer": "John Doe",
  "items": [
    "Laptop",
    "Mouse"
  ],
  "total": 1299.99,
  "status": "shipped",
  "tracking": "TRACK123456"
}

👤 Customer (Turn 2): I want to return it because the laptop arrived with a scratch
🤖 Agent: I'm sorry to hear that your laptop arrived with a scratch, and I appreciate you bringing this to my attention. 

Since the order has already been shipped, I can assist you with processing a refund for the order. The total amount for your order is $1299.99. 

Could you please confirm that you would like to proceed with a refund of $1299.99 for the reason of "received damaged item"?

👤 Customer (Turn 

In [21]:
# Quick individual tests

# Quick test AutoGen only
async def test_autogen():
    await run_autogen_multiturn()

# Quick test Agent Framework only  
async def test_agent_framework():
    await run_agent_framework_multiturn()

# Quick test single-turn only
async def test_single_turn():
    await compare_single_turn()

print("✅ Quick test functions ready!")
print("\n🧪 Available tests:")
print("   • await test_autogen()")
print("   • await test_agent_framework()")
print("   • await test_single_turn()")


✅ Quick test functions ready!

🧪 Available tests:
   • await test_autogen()
   • await test_agent_framework()
   • await test_single_turn()


#### Multi-Agent

##### AI Research Report Generator

In [ ]:
# Install required packages
! pip install autogen-agentchat autogen-ext[openai] agent-framework openai python-dotenv -q
print("✅ All packages installed successfully!")


✅ All packages installed successfully!


In [ ]:
# Import all required libraries
import asyncio
import os
from typing import List, Dict, Optional
from datetime import datetime
from dotenv import load_dotenv

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [3]:
load_dotenv()

True

In [4]:
# Configure API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

print("✅ API Key configured!")


✅ API Key configured!


In [5]:
# Define tools that agents can use

import json

def search_academic_papers(topic: str, max_results: int = 5) -> str:
    """Search for academic papers on a topic (simulated)."""
    print(f"🔍 Searching academic papers about: {topic}")
    
    # Simulated search results
    papers = {
        "artificial intelligence": [
            {"title": "Deep Learning Revolution", "year": 2023, "citations": 450},
            {"title": "Neural Networks Explained", "year": 2024, "citations": 320},
            {"title": "AI in Healthcare", "year": 2023, "citations": 280},
        ],
        "climate change": [
            {"title": "Global Warming Trends", "year": 2024, "citations": 520},
            {"title": "Renewable Energy Solutions", "year": 2023, "citations": 410},
            {"title": "Carbon Capture Technology", "year": 2024, "citations": 380},
        ],
        "quantum computing": [
            {"title": "Quantum Supremacy Achieved", "year": 2024, "citations": 600},
            {"title": "Quantum Algorithms", "year": 2023, "citations": 390},
            {"title": "Quantum Error Correction", "year": 2024, "citations": 350},
        ]
    }
    
    # Find relevant papers
    results = []
    for key in papers:
        if key in topic.lower():
            results = papers[key][:max_results]
            break
    
    if not results:
        results = [{"title": "General Research Paper", "year": 2024, "citations": 100}]
    
    return json.dumps({
        "topic": topic,
        "papers_found": len(results),
        "papers": results
    }, indent=2)


def get_statistics(topic: str) -> str:
    """Get relevant statistics about a topic (simulated)."""
    print(f"📊 Fetching statistics for: {topic}")
    
    stats = {
        "artificial intelligence": {
            "market_size_2024": "$196.63 billion",
            "growth_rate": "37.3% CAGR",
            "key_industries": ["Healthcare", "Finance", "Automotive"]
        },
        "climate change": {
            "global_temp_increase": "1.1°C since pre-industrial",
            "co2_levels": "419 ppm",
            "renewable_adoption": "30% of global energy"
        },
        "quantum computing": {
            "market_size_2024": "$1.3 billion",
            "expected_2030": "$7.6 billion",
            "active_qubits": "1,000+ (IBM Quantum)"
        }
    }
    
    result = stats.get(topic.lower(), {"info": "Limited statistics available"})
    
    return json.dumps({
        "topic": topic,
        "statistics": result
    }, indent=2)


def fact_check(statement: str) -> str:
    """Fact-check a statement (simulated)."""
    print(f"✓ Fact-checking: {statement[:50]}...")
    
    # Simulated fact-checking
    confidence = 0.85
    
    return json.dumps({
        "statement": statement[:100] + "..." if len(statement) > 100 else statement,
        "verified": True,
        "confidence": confidence,
        "sources_checked": 3,
        "verdict": "Statement appears accurate based on available sources"
    }, indent=2)


def check_plagiarism(text: str) -> str:
    """Check text for plagiarism (simulated)."""
    print(f"🔍 Checking plagiarism for text ({len(text)} characters)...")
    
    # Simulated plagiarism check
    similarity = 0.12  # 12% similarity (acceptable)
    
    return json.dumps({
        "text_length": len(text),
        "similarity_score": similarity,
        "status": "PASS" if similarity < 0.25 else "FAIL",
        "message": "Text is original" if similarity < 0.25 else "High similarity detected"
    }, indent=2)


def save_report(content: str, filename: str = "research_report.txt") -> str:
    """Save the final report to a file."""
    print(f"💾 Saving report to: {filename}")
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
        f.write(f"\n\n---\nGenerated at: {datetime.now().isoformat()}")
    
    return json.dumps({
        "status": "success",
        "filename": filename,
        "size_bytes": len(content),
        "timestamp": datetime.now().isoformat()
    }, indent=2)


print("✅ Business functions defined!")
print("\n📋 Available tools:")
print("   • search_academic_papers")
print("   • get_statistics")
print("   • fact_check")
print("   • check_plagiarism")
print("   • save_report")


✅ Business functions defined!

📋 Available tools:
   • search_academic_papers
   • get_statistics
   • fact_check
   • check_plagiarism
   • save_report


In [6]:
# AutoGen - RoundRobin Multi-Agent Research System

async def run_autogen_research_system(topic: str = "artificial intelligence"):
    """
    AutoGen implementation of research report generator.
    Uses RoundRobinGroupChat for automatic agent orchestration.
    """
    from autogen_agentchat.agents import AssistantAgent
    from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
    from autogen_agentchat.teams import RoundRobinGroupChat
    from autogen_ext.models.openai import OpenAIChatCompletionClient
    
    print("\n" + "="*70)
    print("🤖 AUTOGEN - RESEARCH REPORT GENERATOR")
    print("="*70)
    print(f"Topic: {topic}\n")
    
    # Create OpenAI client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    )
    
    # Agent 1: Research Agent
    researcher = AssistantAgent(
        name="researcher",
        model_client=model_client,
        tools=[search_academic_papers, get_statistics],
        system_message="""You are a research specialist.

Your responsibilities:
1. Search for academic papers on the given topic
2. Gather relevant statistics and data
3. Provide comprehensive research findings
4. Cite sources and data points

Format your research clearly with:
- Key findings
- Academic sources
- Statistical data
- Recent developments

Keep it factual and well-organized."""
    )
    
    # Agent 2: Writer Agent
    writer = AssistantAgent(
        name="writer",
        model_client=model_client,
        system_message="""You are a professional technical writer.

Your responsibilities:
1. Take research findings and create a well-structured report
2. Write in clear, engaging prose
3. Organize information logically
4. Create sections: Introduction, Key Findings, Analysis, Conclusion
5. Keep it concise (300-500 words)

Writing style:
- Professional but accessible
- Clear structure with sections
- Evidence-based statements
- Engaging narrative flow"""
    )
    
    # Agent 3: Fact Checker
    fact_checker = AssistantAgent(
        name="fact_checker",
        model_client=model_client,
        tools=[fact_check],
        system_message="""You are a meticulous fact-checker.

Your responsibilities:
1. Verify key statements in the report
2. Check data accuracy
3. Identify any unsupported claims
4. Flag potential issues

If everything checks out, provide your verification.
If issues found, specify what needs correction."""
    )
    
    # Agent 4: Quality Assurance / Editor
    editor = AssistantAgent(
        name="editor",
        model_client=model_client,
        tools=[check_plagiarism, save_report],
        system_message="""You are the final quality assurance editor.

Your responsibilities:
1. Review the complete report for quality
2. Check plagiarism
3. Ensure all sections are complete
4. Verify fact-checker's approval
5. Save the final report if everything is good

If satisfied with quality:
- Save the report
- Type "APPROVED" to end the workflow

If revisions needed:
- Specify exactly what needs improvement
- DO NOT approve yet"""
    )
    
    # Create Round-Robin team
    team = RoundRobinGroupChat(
        participants=[researcher, writer, fact_checker, editor],
        termination_condition=TextMentionTermination("APPROVED") | MaxMessageTermination(15),
    )
    
    # Run the team
    print("🚀 Starting multi-agent workflow...\n")
    print("-"*70)
    
    result = await team.run(task=f"Create a comprehensive research report about {topic}")
    
    print("\n" + "-"*70)
    print("✅ AutoGen workflow completed!")
    print(f"📊 Total messages: {len(result.messages)}")
    
    return result


print("✅ AutoGen research system defined!")
print("🚀 Run with: await run_autogen_research_system('artificial intelligence')")


✅ AutoGen research system defined!
🚀 Run with: await run_autogen_research_system('artificial intelligence')


In [7]:
# Agent Framework - Sequential Pipeline

async def run_agent_framework_sequential(topic: str = "quantum computing"):
    """
    Agent Framework Sequential implementation.
    Linear pipeline: research → write → fact-check → edit
    """
    from agent_framework import AgentRunUpdateEvent, SequentialBuilder
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("⚡ AGENT FRAMEWORK - SEQUENTIAL RESEARCH PIPELINE")
    print("="*70)
    print(f"Topic: {topic}\n")
    
    # Create client
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    
    # Create specialized agents
    researcher = client.create_agent(
        name="researcher",
        instructions="""You are a research specialist.

Search for academic papers and statistics about the given topic.
Provide comprehensive research findings with sources and data.

Format your output with:
- Key research findings
- Academic paper citations
- Statistical data
- Recent developments""",
        tools=[search_academic_papers, get_statistics],
        model="gpt-4o-mini"
    )
    
    writer = client.create_agent(
        name="writer",
        instructions="""You are a professional technical writer.

Take the research findings and create a well-structured report with:
- Introduction
- Key Findings
- Analysis
- Conclusion

Write 300-500 words, professional but accessible.""",
        model="gpt-4o-mini"
    )
    
    fact_checker = client.create_agent(
        name="fact_checker",
        instructions="""You are a fact-checker.

Review the report and verify key statements using the fact_check tool.
Confirm accuracy or flag issues.""",
        tools=[fact_check],
        model="gpt-4o-mini"
    )
    
    editor = client.create_agent(
        name="editor",
        instructions="""You are the quality assurance editor.

Review the report, check for plagiarism, and save it.
Provide final quality assessment.""",
        tools=[check_plagiarism, save_report],
        model="gpt-4o-mini"
    )
    
    # Create sequential workflow
    workflow = SequentialBuilder().participants([researcher, writer, fact_checker, editor]).build()
    
    # Run workflow with streaming
    print("🚀 Starting sequential pipeline...\n")
    print("-"*70)
    
    current_executor = None
    full_output = []
    
    async for event in workflow.run_stream(f"Create a research report about {topic}"):
        if isinstance(event, AgentRunUpdateEvent):
            # Print agent name when switching
            if current_executor != event.executor_id:
                if current_executor is not None:
                    print("\n")
                print(f"\n{'='*70}")
                print(f"🔹 {event.executor_id.upper()}")
                print("="*70)
                current_executor = event.executor_id
            
            if event.data:
                print(event.data.text, end="", flush=True)
                full_output.append(event.data.text)
    
    print("\n" + "-"*70)
    print("✅ Sequential pipeline completed!")
    
    return "".join(full_output)


print("✅ Agent Framework sequential system defined!")
print("🚀 Run with: await run_agent_framework_sequential('quantum computing')")


✅ Agent Framework sequential system defined!
🚀 Run with: await run_agent_framework_sequential('quantum computing')


In [8]:
# Agent Framework - Advanced Workflow with Approval Cycles

async def run_agent_framework_with_approval_loop(topic: str = "climate change"):
    """
    Agent Framework with cyclic workflow.
    Includes approval loop: if quality check fails, goes back to writer.
    """
    from agent_framework import (
        AgentExecutorRequest,
        AgentExecutorResponse,
        AgentRunUpdateEvent,
        WorkflowBuilder,
        WorkflowContext,
        WorkflowOutputEvent,
        executor,
    )
    from agent_framework.openai import OpenAIChatClient
    
    print("\n" + "="*70)
    print("⚡ AGENT FRAMEWORK - WORKFLOW WITH APPROVAL CYCLES")
    print("="*70)
    print(f"Topic: {topic}\n")
    
    # Create client
    client = OpenAIChatClient(model_id="gpt-4o-mini")
    
    # Create agents
    researcher = client.create_agent(
        name="researcher",
        instructions="""You are a research specialist.

Search for academic papers and gather statistics about the topic.
Provide comprehensive research findings with sources.

Be thorough and cite your sources.""",
        tools=[search_academic_papers, get_statistics],
        model="gpt-4o-mini"
    )
    
    writer = client.create_agent(
        name="writer",
        instructions="""You are a professional writer.

Create a well-structured research report with:
- Introduction
- Key Findings (with data)
- Analysis
- Conclusion

Keep it 300-500 words, professional and engaging.""",
        model="gpt-4o-mini"
    )
    
    fact_checker = client.create_agent(
        name="fact_checker",
        instructions="""You are a fact-checker.

Verify key statements in the report using the fact_check tool.
Provide a verdict: VERIFIED or NEEDS_REVISION""",
        tools=[fact_check],
        model="gpt-4o-mini"
    )
    
    quality_checker = client.create_agent(
        name="quality_checker",
        instructions="""You are a quality assurance specialist.

Review the report for:
1. Completeness (all sections present)
2. Clarity and readability
3. Proper structure
4. Professional tone

End your review with either:
- "QUALITY_APPROVED" if the report meets standards
- "NEEDS_REVISION: [specific issues]" if improvements needed""",
        model="gpt-4o-mini"
    )
    
    final_editor = client.create_agent(
        name="final_editor",
        instructions="""You are the final editor.

The report has been approved. Perform final checks:
1. Check plagiarism
2. Save the report
3. Provide completion summary

Use the tools to complete these tasks.""",
        tools=[check_plagiarism, save_report],
        model="gpt-4o-mini"
    )
    
    # Custom executor: Quality approval decision point
    @executor
    async def quality_approval_gate(
        response: AgentExecutorResponse, 
        context: WorkflowContext[AgentExecutorRequest, str]
    ) -> None:
        """Decision node: Check if quality is approved."""
        assert response.full_conversation is not None
        last_message = response.full_conversation[-1]
        
        if "QUALITY_APPROVED" in last_message.text:
            # Quality approved - proceed to final editor
            print("\n\n✅ Quality gate PASSED - Proceeding to final editor")
            await context.send_message(
                AgentExecutorRequest(
                    messages=response.full_conversation, 
                    should_respond=True
                )
            )
        else:
            # Quality issues - send back to writer with feedback
            print("\n\n⚠️  Quality gate FAILED - Sending back to writer for revision")
            revision_request = f"Please revise the report based on this feedback:\n{last_message.text}"
            
            # Create message for writer with revision instructions
            await context.send_message(
                AgentExecutorRequest(
                    messages=[{"role": "user", "content": revision_request}],
                    should_respond=True
                )
            )
    
    # Custom executor: Final completion
    @executor
    async def complete_workflow(
        response: AgentExecutorResponse,
        context: WorkflowContext[AgentExecutorRequest, str]
    ) -> None:
        """Final node: Output the result."""
        assert response.full_conversation is not None
        await context.yield_output("✅ Research report generation completed successfully!")
    
    # Build workflow with cycles
    workflow = (
        WorkflowBuilder()
        # Main flow
        .add_edge(researcher, writer)
        .add_edge(writer, fact_checker)
        .add_edge(fact_checker, quality_checker)
        .add_edge(quality_checker, quality_approval_gate)
        
        # Approval loop: if quality check fails, go back to writer
        .add_edge(quality_approval_gate, writer)  # Creates cycle for revision
        
        # Success path: quality approved → final editor → complete
        .add_edge(quality_approval_gate, final_editor)
        .add_edge(final_editor, complete_workflow)
        
        # Set starting point
        .set_start_executor(researcher)
        .build()
    )
    
    # Run workflow
    print("🚀 Starting workflow with approval cycles...\n")
    print("-"*70)
    
    current_executor = None
    revision_count = 0
    
    async for event in workflow.run_stream(f"Create a research report about {topic}"):
        if isinstance(event, WorkflowOutputEvent):
            print(f"\n\n{'='*70}")
            print("🎉 WORKFLOW COMPLETED")
            print("="*70)
            print(event.data)
            print(f"📊 Total revisions: {revision_count}")
            
        elif isinstance(event, AgentRunUpdateEvent):
            # Track revisions
            if event.executor_id == "writer" and current_executor != "researcher":
                revision_count += 1
            
            # Print agent header when switching
            if current_executor != event.executor_id:
                if current_executor is not None:
                    print("\n")
                print(f"\n{'='*70}")
                print(f"🔹 {event.executor_id.upper()}")
                print("="*70)
                current_executor = event.executor_id
            
            if event.data:
                print(event.data.text, end="", flush=True)
    
    print("\n" + "-"*70)
    print("✅ Cyclic workflow completed!")


print("✅ Agent Framework with approval cycles defined!")
print("🚀 Run with: await run_agent_framework_with_approval_loop('climate change')")


✅ Agent Framework with approval cycles defined!
🚀 Run with: await run_agent_framework_with_approval_loop('climate change')


In [9]:
# Display comparison of all three approaches

def show_comparison():
    """Display comprehensive comparison."""
    
    comparison = """
╔═══════════════════════════════════════════════════════════════════════════╗
║           MULTI-AGENT RESEARCH SYSTEM COMPARISON                          ║
╚═══════════════════════════════════════════════════════════════════════════╝

📊 IMPLEMENTATION COMPARISON
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Approach                  | AutoGen          | AF Sequential    | AF Workflow
                          | RoundRobin       |                  | (with cycles)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Setup Complexity          | Low              | Very Low         | Moderate
Code Lines                | ~80              | ~60              | ~150
Revision Cycles           | ✅ Automatic     | ❌ None          | ✅ Custom logic
Agent Coordination        | ✅ Automatic     | ✅ Sequential    | ✅ Graph-based
Termination Control       | ✅ Conditions    | ⚠️  End of seq   | ✅ Custom gates
Conditional Branching     | ⚠️  Limited      | ❌ None          | ✅ Full control
Quality Gates             | ⚠️  Via message  | ❌ None          | ✅ Native
Real-time Streaming       | ✅ Yes           | ✅ Yes           | ✅ Yes
Learning Curve            | Medium           | Low              | Medium-High
Best For                  | Quick prototype  | Simple pipeline  | Production


🎯 USE CASE ANALYSIS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Research Report Generation:
├─ AutoGen RoundRobin:
│  ✅ Great for iterative refinement
│  ✅ Natural conversation flow
│  ⚠️  Less control over exact flow
│
├─ Agent Framework Sequential:
│  ✅ Perfect for first draft generation
│  ✅ Fast and predictable
│  ❌ Can't handle revisions
│
└─ Agent Framework Workflow:
   ✅ Best for production systems
   ✅ Quality gates and approval loops
   ✅ Audit trail and control
   ⚠️  More setup required


🔄 WORKFLOW PATTERNS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

AutoGen RoundRobin:
    Research → Write → Fact-Check → Edit
       ↑__________________________|
    (loops automatically until "APPROVED")

Agent Framework Sequential:
    Research → Write → Fact-Check → Edit → END
    (no loops, single pass)

Agent Framework Workflow:
    Research → Write → Fact-Check → Quality Gate
                  ↑                      |
                  |_____FAIL_____________|
                  |
                  PASS
                  ↓
            Final Edit → Complete


💡 DECISION GUIDE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Choose AutoGen RoundRobin when:
✓ Building prototype quickly
✓ Want automatic revision loops
✓ Don't need complex branching
✓ Agents should all participate equally

Choose Agent Framework Sequential when:
✓ Simple one-pass workflow
✓ No revisions needed
✓ Performance is critical
✓ Building data processing pipelines

Choose Agent Framework Workflow when:
✓ Need quality gates and approvals
✓ Complex branching logic required
✓ Production system with audit needs
✓ Want full control over flow
✓ Building enterprise applications


📋 REAL-WORLD APPLICATIONS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Research Report Generation:
  Best: AF Workflow (quality gates needed)

Content Marketing Pipeline:
  Best: AutoGen (iterative refinement)

Document Processing:
  Best: AF Sequential (fast, predictable)

Code Review System:
  Best: AF Workflow (approval loops, gates)

Customer Support Escalation:
  Best: AF Workflow (conditional routing)

Blog Post Creation:
  Best: AutoGen (conversational iteration)

Data Analysis Report:
  Best: AF Sequential (linear process)

Insurance Claim Processing:
  Best: AF Workflow (decision points)
"""
    
    print(comparison)

show_comparison()



╔═══════════════════════════════════════════════════════════════════════════╗
║           MULTI-AGENT RESEARCH SYSTEM COMPARISON                          ║
╚═══════════════════════════════════════════════════════════════════════════╝

📊 IMPLEMENTATION COMPARISON
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Approach                  | AutoGen          | AF Sequential    | AF Workflow
                          | RoundRobin       |                  | (with cycles)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Setup Complexity          | Low              | Very Low         | Moderate
Code Lines                | ~80              | ~60              | ~150
Revision Cycles           | ✅ Automatic     | ❌ None          | ✅ Custom logic
Agent Coordination        | ✅ Automatic     | ✅ Sequential    | ✅ Graph-based
Termination Control       | ✅ Conditions    | ⚠️  End of seq   | ✅ Custom gates
Conditional Branching     | ⚠️  Limited   

In [10]:
# Run AutoGen Research System
await run_autogen_research_system("artificial intelligence")


🤖 AUTOGEN - RESEARCH REPORT GENERATOR
Topic: artificial intelligence

🚀 Starting multi-agent workflow...

----------------------------------------------------------------------
🔍 Searching academic papers about: artificial intelligence
📊 Fetching statistics for: artificial intelligence
✓ Fact-checking: The market size of AI is projected to reach $196.6...✓ Fact-checking: The growth rate of AI is 37.3% CAGR....

✓ Fact-checking: The paper 'Deep Learning Revolution' has 450 citat...
✓ Fact-checking: The paper 'Neural Networks Explained' has 320 cita...
✓ Fact-checking: The paper 'AI in Healthcare' has 280 citations....
🔍 Checking plagiarism for text (3787 characters)...💾 Saving report to: artificial_intelligence_report.txt

💾 Saving report to: artificial_intelligence_report.txt

----------------------------------------------------------------------
✅ AutoGen workflow completed!
📊 Total messages: 21


TaskResult(messages=[TextMessage(id='a17209c9-9034-4e70-8669-3ad833b6e17f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 12, 17, 18, 49, 25, 265999, tzinfo=datetime.timezone.utc), content='Create a comprehensive research report about artificial intelligence', type='TextMessage'), ToolCallRequestEvent(id='3a55a9f3-8d28-44c5-b134-5375eedfd774', source='researcher', models_usage=RequestUsage(prompt_tokens=187, completion_tokens=56), metadata={}, created_at=datetime.datetime(2025, 12, 17, 18, 49, 27, 622743, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_L5iaSieJMdG9XRax2nlFtLPl', arguments='{"topic": "artificial intelligence", "max_results": 5}', name='search_academic_papers'), FunctionCall(id='call_3UuDer0WFKRWvRO8FZzpfhaO', arguments='{"topic": "artificial intelligence"}', name='get_statistics')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='fd1879cf-3876-48c5-bb80-eb8113ad6d91', source='researcher', models_usage=None, meta

In [11]:
# Run Agent Framework Sequential
await run_agent_framework_sequential("quantum computing")


⚡ AGENT FRAMEWORK - SEQUENTIAL RESEARCH PIPELINE
Topic: quantum computing



Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Addi

🚀 Starting sequential pipeline...

----------------------------------------------------------------------

🔹 RESEARCHER
🔍 Searching academic papers about: quantum computing
📊 Fetching statistics for: quantum computing
# Research Report on Quantum Computing

## Key Research Findings

1. **Quantum Supremacy**: A recent landmark achievement in 2024 titled "Quantum Supremacy Achieved" has demonstrated that quantum computers can perform calculations that are infeasible for classical computers. This paper has garnered significant attention, with over 600 citations, underscoring its impact on the field.

2. **Quantum Algorithms**: The study "Quantum Algorithms" published in 2023 discusses various algorithms that leverage quantum mechanics principles, improving computational efficiency for specific problem sets. This research has accumulated around 390 citations, reflecting its relevance in practical quantum computing applications.

3. **Quantum Error Correction**: In 2024, the paper titled "Q

'# Research Report on Quantum Computing\n\n## Key Research Findings\n\n1. **Quantum Supremacy**: A recent landmark achievement in 2024 titled "Quantum Supremacy Achieved" has demonstrated that quantum computers can perform calculations that are infeasible for classical computers. This paper has garnered significant attention, with over 600 citations, underscoring its impact on the field.\n\n2. **Quantum Algorithms**: The study "Quantum Algorithms" published in 2023 discusses various algorithms that leverage quantum mechanics principles, improving computational efficiency for specific problem sets. This research has accumulated around 390 citations, reflecting its relevance in practical quantum computing applications.\n\n3. **Quantum Error Correction**: In 2024, the paper titled "Quantum Error Correction" addresses the methods for mitigating errors in quantum computations, a critical challenge affecting the reliability of quantum systems. The paper has been cited about 350 times, showin

In [12]:
# Run complete comparison

async def run_all_comparisons():
    """Run all three implementations for comparison."""
    
    print("🎯 COMPLETE MULTI-AGENT COMPARISON")
    print("="*70)
    print("\nWe'll create research reports using three different approaches:\n")
    
    # 1. AutoGen
    print("\n1️⃣  AUTOGEN ROUNDROBIN APPROACH")
    await run_autogen_research_system("artificial intelligence")
    
    input("\n⏸️  Press Enter to continue to Sequential approach...")
    
    # 2. Sequential
    print("\n\n2️⃣  AGENT FRAMEWORK SEQUENTIAL APPROACH")
    await run_agent_framework_sequential("quantum computing")
    
    input("\n⏸️  Press Enter to continue to Workflow approach...")
    
    # 3. Workflow with cycles
    print("\n\n3️⃣  AGENT FRAMEWORK WORKFLOW (WITH CYCLES)")
    await run_agent_framework_with_approval_loop("climate change")
    
    print("\n\n" + "="*70)
    print("✅ ALL COMPARISONS COMPLETED!")
    print("="*70)
    print("\n📊 Summary:")
    print("   • AutoGen: Easy setup, automatic coordination")
    print("   • Sequential: Fast, simple, no revisions")
    print("   • Workflow: Maximum control, quality gates, production-ready")
    print("\n💡 Choose based on your needs for control vs simplicity!\n")

print("✅ Complete comparison ready!")
print("🚀 Run with: await run_all_comparisons()")


✅ Complete comparison ready!
🚀 Run with: await run_all_comparisons()


In [13]:
# Test with your own topics

async def generate_report_custom(topic: str, approach: str = "autogen"):
    """
    Generate a research report on any topic.
    
    Args:
        topic: The research topic
        approach: "autogen", "sequential", or "workflow"
    """
    print(f"\n🎯 Generating report about: {topic}")
    print(f"📋 Using approach: {approach.upper()}\n")
    
    if approach.lower() == "autogen":
        await run_autogen_research_system(topic)
    elif approach.lower() == "sequential":
        await run_agent_framework_sequential(topic)
    elif approach.lower() == "workflow":
        await run_agent_framework_with_approval_loop(topic)
    else:
        print("❌ Invalid approach. Choose: autogen, sequential, or workflow")


print("✅ Custom topic generator ready!")
print("\n🧪 Examples:")
print('   await generate_report_custom("blockchain technology", "autogen")')
print('   await generate_report_custom("space exploration", "sequential")')
print('   await generate_report_custom("renewable energy", "workflow")')


✅ Custom topic generator ready!

🧪 Examples:
   await generate_report_custom("blockchain technology", "autogen")
   await generate_report_custom("space exploration", "sequential")
   await generate_report_custom("renewable energy", "workflow")


In [14]:
# Generate report on your topic

# Try different topics and approaches!
await generate_report_custom("machine learning", "workflow")

# Or try:
# await generate_report_custom("blockchain", "autogen")
# await generate_report_custom("biotechnology", "sequential")



🎯 Generating report about: machine learning
📋 Using approach: WORKFLOW


⚡ AGENT FRAMEWORK - WORKFLOW WITH APPROVAL CYCLES
Topic: machine learning



Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Adding an edge with Executor or AgentProtocol instances directly is not recommended, because workflow instances created from the builder will share the same executor/agent instances. Consider using a registered name for lazy initialization instead.
Addi

🚀 Starting workflow with approval cycles...

----------------------------------------------------------------------
🔍 Searching academic papers about: machine learning
📊 Fetching statistics for: machine learning

🔹 RESEARCHER
### Research Report on Machine Learning

**Introduction**

Machine learning (ML) is a pivotal subset of artificial intelligence (AI), enabling systems to learn from data, identify patterns, and make decisions with minimal human intervention. The field has experienced rapid growth and transformative applications across various industries, such as healthcare, finance, and autonomous systems.

---

**Academic Contributions**

Recent scholarly contributions to the field of machine learning include:

1. **Title**: General Research Paper
   - **Year**: 2024
   - **Citations**: 100

   This research paper highlights advancements in algorithms and applications of machine learning, emphasizing the integration of ML techniques in modern technological landscapes. (Note: Deta

ServiceResponseException: <class 'agent_framework.openai._chat_client.OpenAIChatClient'> service failed to complete the prompt: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-HGSv0FzyPfsnL2ruOd42SEDo on tokens per min (TPM): Limit 200000, Used 184081, Requested 75337. Please try again in 17.825s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

💾 Saving report to: machine_learning_research_report.txt
